In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential
from keras import metrics,layers
import pydicom
import cv2
import glob
import os
import gc
import matplotlib.pyplot as plt
%matplotlib inline

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
print(train.shape)
train.head()

In [ ]:
train['study'] = train['BraTS21ID'].apply(lambda x: '0'*(5-len(str(x))) + str(x))
train.tail()

In [ ]:
X = train.study.values
y = train.MGMT_value.values

In [ ]:
def get_array(batch_x,SERIES='FLAIR',kind='train'):
    ARRAY = np.empty((0,5,64,64,1))
    SERIES= SERIES
    bs = len(batch_x)
    for i in range(bs):
        study = batch_x[i]
        array = np.empty((0,64,64,1))
        file_names = glob.glob(f"../input/rsna-miccai-brain-tumor-radiogenomic-classification/{kind}/" + str(study) + '/' + str(SERIES) + '/*')
        file_names.sort()
        file_names = file_names[-5:]
        
        for fn in file_names:
            img = pydicom.dcmread(fn).pixel_array
            img = ((img - img.min())/(img.max()+1e-4)).astype(float)
            img = cv2.resize(img,(64,64))
            array = np.append(array,img.reshape(1,64,64,1),axis=0)
        ARRAY = np.append(ARRAY,array.reshape(1,5,64,64,1),axis=0)
    
    return ARRAY
    

In [ ]:
class RSNADataset(tf.keras.utils.Sequence):
    def __init__(self, study, target, batch_size=8,test=0):
        self.study, self.target = study, target
        self.batch_size = batch_size
        self.test = test

    def __len__(self):
        return int(np.ceil(len(self.study) / float(self.batch_size)))

    def __getitem__(self, idx):
        
        if self.test==0:
            batch_x = self.study[idx * self.batch_size:(idx + 1) * self.batch_size]
            FLAIR_ARRAY = get_array(batch_x,'FLAIR','train')
            T1W_ARRAY = get_array(batch_x,'T1w','train')
            T1WCE_ARRAY = get_array(batch_x,'T1wCE','train')
            T2W_ARRAY = get_array(batch_x,'T2w','train')
            batch_y = self.target[idx * self.batch_size:(idx + 1) * self.batch_size]
            
            return [FLAIR_ARRAY,T1W_ARRAY,T1WCE_ARRAY,T2W_ARRAY],[np.array(batch_y)]
        
        
        else:
            batch_x = self.study[idx * self.batch_size:(idx + 1) * self.batch_size]
            FLAIR_ARRAY = get_array(batch_x,'FLAIR','test')
            T1W_ARRAY = get_array(batch_x,'T1w','test')
            T1WCE_ARRAY = get_array(batch_x,'T1wCE','test')
            T2W_ARRAY = get_array(batch_x,'T2w','test')
            
            return [FLAIR_ARRAY,T1W_ARRAY,T1WCE_ARRAY,T2W_ARRAY]

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=2021)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
train_ds = RSNADataset(X_train,y_train)
val_ds = RSNADataset(X_test,y_test)

In [ ]:
def get_model():
    base_model = tf.keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(64,64,3))
    
    input1 = layers.Input(shape=(5,64,64,1))
    inp1 = layers.Concatenate(axis=-1)([input1,input1,input1])

    input2 = layers.Input(shape=(5,64,64,1))
    inp2 = layers.Concatenate(axis=-1)([input2,input2,input2])

    input3 = layers.Input(shape=(5,64,64,1))
    inp3 = layers.Concatenate(axis=-1)([input3,input3,input3])

    input4 = layers.Input(shape=(5,64,64,1))
    inp4 = layers.Concatenate(axis=-1)([input4,input4,input4])
    
    time_dist_layer = Sequential(
                        [base_model,
                        layers.GlobalAveragePooling2D(),
                        ]
                        ) 
    
    
    x1 = tf.keras.layers.TimeDistributed(time_dist_layer)(inp1)
    x2 = tf.keras.layers.TimeDistributed(time_dist_layer)(inp2)
    x3 = tf.keras.layers.TimeDistributed(time_dist_layer)(inp3)
    x4 = tf.keras.layers.TimeDistributed(time_dist_layer)(inp4)

    #x = x1
    x = layers.Concatenate(axis=-1)([x1,x2,x3,x4])
    
    x = layers.LSTM(256)(x)
    out = layers.Dense(1,activation='sigmoid')(x)
    model = tf.keras.Model(inputs=[input1,input2,input3,input4], outputs=out)
    return model
    
    

In [ ]:
model = get_model()
model.summary()

In [ ]:
model.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy(),metrics=tf.keras.metrics.AUC())

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/model.h5',monitor='val_loss',mode='min',save_best_only=True)

In [ ]:
model.fit(train_ds,validation_data=val_ds,epochs=10,callbacks=model_checkpoint_callback)

In [ ]:
test = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
print(test.shape)
test.head()

In [ ]:
test['study'] = test['BraTS21ID'].apply(lambda x: '0'*(5-len(str(x))) + str(x))
test.tail()

In [ ]:
test_ds = RSNADataset(test.study.values,None,test=1)

In [ ]:
model = tf.keras.models.load_model("/kaggle/working/model.h5")

preds = model.predict(test_ds)

In [ ]:
test['MGMT_value'] = preds
test.head()

In [ ]:
test[['BraTS21ID','MGMT_value']].to_csv('submission.csv',index=False)